In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalMaxPool1D, Conv1D, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# ====== Load and Preprocess Arabic Dataset ======
df = pd.read_csv("Arabic_cleaned.csv")

# Drop any rows with missing or empty cleanedtext
df['cleanedtext'] = df['cleanedtext'].astype(str).str.strip()
df = df[df['cleanedtext'] != '']
df = df.dropna(subset=['cleanedtext'])

# ====== Tokenization and Padding ======
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleanedtext'])
X = tokenizer.texts_to_sequences(df['cleanedtext'])
X = pad_sequences(X, maxlen=max_len)

# ====== Labels ======
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])  # Encode -1 and 1 to 0 and 1
y_cat = to_categorical(y)

# ====== Train-test Split ======
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42, stratify=y_cat)

# ====== Early Stopping ======
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# ====== Models ======
def create_dense_model():
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
        GlobalMaxPool1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_model():
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_bilstm_model():
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
        Bidirectional(LSTM(64)),
        Dropout(0.5),
        Dense(32, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ====== Training Function ======
def train_and_evaluate(model_fn, name):
    print(f"\nTraining {name}...")
    model = model_fn()
    model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1, callbacks=[early_stop], verbose=2)

    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f"{name} Accuracy: {acc:.4f}")
    print(f"{name} F1 Score: {f1:.4f}")
    return model, acc, f1

# ====== Train All Models and Save the Best ======
models = [
    ("Dense", create_dense_model),
    ("CNN", create_cnn_model),
    ("BiLSTM", create_bilstm_model)
]

results = []
best_model = None
best_score = 0

for name, fn in models:
    model, acc, f1 = train_and_evaluate(fn, name)
    results.append((name, acc, f1))
    if acc > best_score:
        best_score = acc
        best_model = model
        best_model.save("best_arabic_model.keras")

# ====== Results Summary ======
print("\n=== Arabic Model Results Summary ===")
for name, acc, f1 in sorted(results, key=lambda x: x[1], reverse=True):
    print(f"{name}: Accuracy = {acc:.4f}, F1 Score = {f1:.4f}")


Training Dense...


e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
375/375 - 9s - 23ms/step - accuracy: 0.7569 - loss: 0.4764 - val_accuracy: 0.8365 - val_loss: 0.3575
Epoch 2/5
375/375 - 6s - 15ms/step - accuracy: 0.8749 - loss: 0.3050 - val_accuracy: 0.8369 - val_loss: 0.3539
Epoch 3/5
375/375 - 6s - 15ms/step - accuracy: 0.9087 - loss: 0.2380 - val_accuracy: 0.8384 - val_loss: 0.3920
Epoch 4/5
375/375 - 6s - 15ms/step - accuracy: 0.9329 - loss: 0.1832 - val_accuracy: 0.8316 - val_loss: 0.4212
417/417 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Dense Accuracy: 0.8479
Dense F1 Score: 0.8477

Training CNN...
Epoch 1/5


e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


375/375 - 34s - 90ms/step - accuracy: 0.7888 - loss: 0.4208 - val_accuracy: 0.8526 - val_loss: 0.3298
Epoch 2/5
375/375 - 31s - 83ms/step - accuracy: 0.8850 - loss: 0.2777 - val_accuracy: 0.8583 - val_loss: 0.3234
Epoch 3/5
375/375 - 31s - 83ms/step - accuracy: 0.9212 - loss: 0.2051 - val_accuracy: 0.8481 - val_loss: 0.3576
Epoch 4/5
375/375 - 32s - 86ms/step - accuracy: 0.9463 - loss: 0.1472 - val_accuracy: 0.8423 - val_loss: 0.4151
417/417 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
CNN Accuracy: 0.8575
CNN F1 Score: 0.8574

Training BiLSTM...
Epoch 1/5


e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


375/375 - 105s - 280ms/step - accuracy: 0.7567 - loss: 0.4652 - val_accuracy: 0.8461 - val_loss: 0.3420
Epoch 2/5
375/375 - 136s - 364ms/step - accuracy: 0.8789 - loss: 0.2889 - val_accuracy: 0.8487 - val_loss: 0.3349
Epoch 3/5
375/375 - 139s - 370ms/step - accuracy: 0.9025 - loss: 0.2378 - val_accuracy: 0.8528 - val_loss: 0.3473
Epoch 4/5
375/375 - 96s - 256ms/step - accuracy: 0.9262 - loss: 0.1890 - val_accuracy: 0.8463 - val_loss: 0.3922
417/417 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step
BiLSTM Accuracy: 0.8522
BiLSTM F1 Score: 0.8515

=== Arabic Model Results Summary ===
CNN: Accuracy = 0.8575, F1 Score = 0.8574
BiLSTM: Accuracy = 0.8522, F1 Score = 0.8515
Dense: Accuracy = 0.8479, F1 Score = 0.8477


In [4]:
def predict_arabic_text(sample_text, model, tokenizer, label_encoder, threshold=0.1):
    sequence = tokenizer.texts_to_sequences([sample_text])
    padded = pad_sequences(sequence, maxlen=max_len)
    pred_prob = model.predict(padded, verbose=0)[0]
    pred_class = np.argmax(pred_prob)
    confidence_gap = sorted(pred_prob, reverse=True)[0] - sorted(pred_prob, reverse=True)[1]

    if confidence_gap < threshold:
        label = "neutral"
    else:
        original_label = label_encoder.inverse_transform([pred_class])[0]
        label = "positive" if original_label == 1 else "negative"

    print(f"Input: {sample_text}")
    print(f"Probabilities: {pred_prob}")
    print(f"Predicted Sentiment: {label}")


In [6]:
from tensorflow.keras.models import load_model

# Load model and test
arabic_model = load_model("best_arabic_model.keras")

test_texts = [
    "الخدمة كانت مقبولة ولكن ليست ممتازة.",
    "هذا أفضل منتج استخدمته في حياتي!",
    "لا أنصح به على الإطلاق، تجربة سيئة.",
    "أداء رائع وسرعة في التوصيل، شكراً لكم!",
    "تعامل راقٍ واحترافي، أنصح بالتجربة.",
    "تجربة سيئة جدًا، المنتج لا يعمل كما يجب.",
    "خدمة سيئة وسعر مرتفع بلا داعٍ.",
    "المنتج متوسط، ليس الأفضل ولا الأسوأ.",
    "المنتج فاق توقعاتي بكل صراحة.",
    "وصل المنتج تالفًا وبحجم مختلف عن المطلوب."
]

for text in test_texts:
    predict_arabic_text(text, arabic_model, tokenizer, label_encoder)
    print("-" * 50)

Input: الخدمة كانت مقبولة ولكن ليست ممتازة.
Probabilities: [0.64340633 0.35659367]
Predicted Sentiment: negative
--------------------------------------------------
Input: هذا أفضل منتج استخدمته في حياتي!
Probabilities: [0.4053409 0.5946591]
Predicted Sentiment: positive
--------------------------------------------------
Input: لا أنصح به على الإطلاق، تجربة سيئة.
Probabilities: [0.8505613  0.14943871]
Predicted Sentiment: negative
--------------------------------------------------
Input: أداء رائع وسرعة في التوصيل، شكراً لكم!
Probabilities: [0.05260722 0.9473928 ]
Predicted Sentiment: positive
--------------------------------------------------
Input: تعامل راقٍ واحترافي، أنصح بالتجربة.
Probabilities: [0.5902418 0.4097582]
Predicted Sentiment: negative
--------------------------------------------------
Input: تجربة سيئة جدًا، المنتج لا يعمل كما يجب.
Probabilities: [0.66115725 0.33884272]
Predicted Sentiment: negative
--------------------------------------------------
Input: خدمة سيئة وسع

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from transformers import AutoTokenizer
import tensorflow as tf

# ====== Load Data ======
df = pd.read_csv("Arabic_cleaned.csv")
df = df[df['cleanedtext'].str.strip() != '']
df = df.dropna(subset=['cleanedtext'])

# Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])  # -1 → 0, 1 → 1
y = to_categorical(df['label'])

# ====== AraBERT Tokenizer ======
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

# Tokenize text
def tokenize(texts, max_len=100):
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,
        padding='max_length',
        max_length=max_len,
        return_tensors='np'
    )
    return encodings['input_ids']

X = tokenize(df['cleanedtext'], max_len=100)

# ====== Train-Test Split ======
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# ====== Early Stopping ======
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# ====== Custom BiLSTM Model ======
vocab_size = tokenizer.vocab_size  # Get AraBERT tokenizer vocab size
embedding_dim = 128
max_len = X.shape[1]

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ====== Train ======
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1, callbacks=[early_stop], verbose=2)

# ====== Evaluate ======
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"\nBiLSTM Accuracy: {acc:.4f}")
print(f"BiLSTM F1 Score: {f1:.4f}")

e:\Social Media Sentiment Analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/5


e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


750/750 - 232s - 309ms/step - accuracy: 0.8179 - loss: 0.3841 - val_accuracy: 0.8556 - val_loss: 0.3224
Epoch 2/5
750/750 - 227s - 303ms/step - accuracy: 0.9079 - loss: 0.2350 - val_accuracy: 0.8507 - val_loss: 0.3596
Epoch 3/5
750/750 - 234s - 312ms/step - accuracy: 0.9525 - loss: 0.1352 - val_accuracy: 0.8419 - val_loss: 0.4458
417/417 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step

BiLSTM Accuracy: 0.8592
BiLSTM F1 Score: 0.8592


In [19]:
def predict2_arabic_text(text, model, tokenizer, le, max_len=100):
    # Tokenize the input text using AraBERT tokenizer
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_len,
        return_tensors='np'
    )

    # Get input_ids
    input_ids = encoding['input_ids']

    # Predict probabilities
    prediction = model.predict(input_ids)

    # Get predicted class index
    pred_class_idx = np.argmax(prediction, axis=1)[0]

    # Decode original label (-1 or 1)
    label = le.inverse_transform([pred_class_idx])[0]

    # Confidence score
    confidence = float(np.max(prediction))

    sentiment = "Positive" if label == 1 else "Negative"

    print(f"Text: {text}")
    print(f"Predicted Sentiment: {sentiment}")
    print(f"Confidence: {confidence:.2f}")

In [20]:
test_texts = [
    "الخدمة كانت مقبولة ولكن ليست ممتازة.",
    "هذا أفضل منتج استخدمته في حياتي!",
    "لا أنصح به على الإطلاق، تجربة سيئة.",
    "أداء رائع وسرعة في التوصيل، شكراً لكم!",
    "تعامل راقٍ واحترافي، أنصح بالتجربة.",
    "تجربة سيئة جدًا، المنتج لا يعمل كما يجب.",
    "خدمة سيئة وسعر مرتفع بلا داعٍ.",
    "المنتج متوسط، ليس الأفضل ولا الأسوأ.",
    "المنتج فاق توقعاتي بكل صراحة.",
    "وصل المنتج تالفًا وبحجم مختلف عن المطلوب."
]

for text in test_texts:
    predict2_arabic_text(text, model, tokenizer, le)
    print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Text: الخدمة كانت مقبولة ولكن ليست ممتازة.
Predicted Sentiment: Negative
Confidence: 0.80
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Text: هذا أفضل منتج استخدمته في حياتي!
Predicted Sentiment: Positive
Confidence: 0.55
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Text: لا أنصح به على الإطلاق، تجربة سيئة.
Predicted Sentiment: Negative
Confidence: 0.85
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Text: أداء رائع وسرعة في التوصيل، شكراً لكم!
Predicted Sentiment: Positive
Confidence: 0.97
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Text: تعامل راقٍ واحترافي، أنصح بالتجربة.
Predicted Sentiment: Positive
Confidence: 0.70
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Text: تجربة سيئة جدًا، المنتج لا يعمل كما يجب.
Predicted Sentiment: Ne

In [21]:
AraBERT_Tokenization_Model = model 
AraBERT_Tokenization_Model.save("araBERT_tokenization_model.keras")

In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# === Load Dataset ===
df = pd.read_csv("Arabic_cleaned.csv")
df = df.dropna(subset=["cleanedtext"])
df["cleanedtext"] = df["cleanedtext"].astype(str)

# === Encode Labels (-1 to 0, 1 to 1) ===
le = LabelEncoder()
df["label"] = le.fit_transform(df["label"])

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(df["cleanedtext"], df["label"], test_size=0.2, random_state=42)

# === Tokenization ===
tokenizer = Tokenizer(num_words=50000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 128
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")

# === Build the Model ===
model = Sequential([
    Embedding(input_dim=50000, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# === Compile ===
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# === Train ===
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.1)

# === Evaluate ===
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Loss: {loss:.4f} - Accuracy: {accuracy*100:.2f}%")

e:\Social Media Sentiment Analysis\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 663s 432ms/step - accuracy: 0.7811 - loss: 0.4331 - val_accuracy: 0.8552 - val_loss: 0.3219
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 640s 427ms/step - accuracy: 0.9259 - loss: 0.1956 - val_accuracy: 0.8519 - val_loss: 0.3628
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 645s 430ms/step - accuracy: 0.9648 - loss: 0.1030 - val_accuracy: 0.8470 - val_loss: 0.4444
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 652s 435ms/step - accuracy: 0.9790 - loss: 0.0631 - val_accuracy: 0.8449 - val_loss: 0.5601
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 649s 432ms/step - accuracy: 0.9891 - loss: 0.0346 - val_accuracy: 0.8397 - val_loss: 0.7729
417/417 ━━━━━━━━━━━━━━━━━━━━ 46s 110ms/step - accuracy: 0.8367 - loss: 0.7640
Test Loss: 0.7531 - Accuracy: 83.95%


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping

from gensim.models import KeyedVectors

# ============================
# Step 1: Load and Clean Data
# ============================

# Replace with your own data
# Dataset should have columns: ['text', 'label'] with label as 0/1
df = pd.read_csv("arabic_sentiment_data.csv")

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

df['text'] = df['text'].astype(str).apply(clean_text)

# ============================
# Step 2: Tokenization
# ============================

max_len = 100
vocab_size = 50000

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df['text'])

X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, maxlen=max_len)

y = df['label'].values

X_train_pad, X_test_pad, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ============================
# Step 3: Load AraVec Embeddings
# ============================

print("Loading AraVec Word2Vec embeddings...")
aravec_path = "aravec_cbow.bin"  # Path to AraVec CBOW binary
aravec_model = KeyedVectors.load_word2vec_format(aravec_path, binary=True)

embedding_dim = 300
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if i >= vocab_size:
        continue
    if word in aravec_model:
        embedding_matrix[i] = aravec_model[word]

# ============================
# Step 4: Build GRU + CNN Model
# ============================

model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              input_length=max_len,
              weights=[embedding_matrix],
              trainable=False),
    Bidirectional(GRU(64, return_sequences=True)),
    Conv1D(128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# ============================
# Step 5: Train Model
# ============================

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    epochs=15,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop]
)

# ============================
# Step 6: Plot Performance
# ============================

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# ============================
# Step 7: Evaluate on Test Set
# ============================

y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))

AttributeError: module 'inspect' has no attribute 'ArgSpec'